In [68]:
import pandas as pd
#import matplotlib.pyplot as plt
#import matplotlib.dates as mdates
import numpy as np
#from datetime import datetime
#from datetime import date

In [69]:
inventory = pd.read_csv('inputs/InventoryList_2023-03-14.csv',header=0,dtype={'ProductCode': str,'Barcode':str})
begin = pd.read_excel('inputs/SbPS June19 Sept19.xlsx',engine='openpyxl',header=0,skiprows=5,dtype={'ProductCode': str,'Barcode':str})
end   = pd.read_excel('inputs/SbPS June22 Sept22.xlsx',engine='openpyxl',header=0,skiprows=5,dtype={'ProductCode': str,'Barcode':str})

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3,25,69,74,79) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [70]:
#inventory.columns
inventory = inventory.drop(columns=['Brand', 'Type', 'FixedAssetType','CostingMethod', 'Length', 'Width', 'Height', 'Weight', 'CartonLength',
       'CartonWidth', 'CartonHeight', 'CartonInnerQuantity', 'CartonQuantity','CartonVolume', 'WeightUnits', 'DimensionUnits','MinimumBeforeReorder', 'ReorderQuantity', 'DefaultLocation',
       'LastSuppliedBy', 'SupplierProductCode', 'SupplierProductName','PriceTier3','PriceTier4', 'PriceTier5', 'PriceTier6', 'PriceTier7', 'PriceTier8',
       'PriceTier9', 'PriceTier10', 'AssemblyBOM', 'AutoAssemble','AutoDisassemble', 'DropShip', 'DropShipSupplier', 'DefaultUnitOfMeasure', 'InventoryAccount', 'RevenueAccount',
       'ExpenseAccount', 'COGSAccount', 'ProductAttributeSet','AdditionalAttribute1', 'AdditionalAttribute2', 'AdditionalAttribute3',
       'AdditionalAttribute4', 'AdditionalAttribute5', 'AdditionalAttribute6','AdditionalAttribute7', 'AdditionalAttribute8', 'AdditionalAttribute9',
       'AdditionalAttribute10', 'DiscountName', 'ProductFamilySKU','ProductFamilyName', 'ProductFamilyOption1Name',
       'ProductFamilyOption1Value', 'ProductFamilyOption2Name','ProductFamilyOption2Value', 'ProductFamilyOption3Name',
       'ProductFamilyOption3Value', 'CommaDelimitedTags', 'StockLocator','PurchaseTaxRule', 'SaleTaxRule', 'Description',
       'ShortDescription', 'Sellable', 'PickZones', 'AlwaysShowQuantity','WarrantySetupName', 'InternalNote', 'ProductionBOM', 'MakeToOrderBom',
       'QuantityToProduce', 'IsAccountingDimensionEnabled','DimensionAttribute1', 'DimensionAttribute2', 'DimensionAttribute3',
       'DimensionAttribute4', 'DimensionAttribute5', 'DimensionAttribute6','DimensionAttribute7', 'DimensionAttribute8', 'DimensionAttribute9',
       'DimensionAttribute10', 'HSCode', 'CountryOfOrigin'])
inventory.head()

,ProductCode,Name,Category,Barcode,SupplierFixedPrice,PriceTier1,PriceTier2,AverageCost,Status
0,NaN,NaN,Service,NaN,0.00,50.00,0.00,0.000,ACTIVE
1,777823111113,1 Water 355ml,Non Alcoholic Beverage,NaN,0.00,1.50,0.00,0.000,ACTIVE
2,'777823111113,1 Water 355ml,Non Alcoholic Beverage,777823111113,0.00,1.50,0.95,0.506,DEPRECATED
3,851233007157,11 Below Brewing Taco Tuesday Cans 6 Pack,Craft Beer,851233007157,7.01,12.99,10.80,8.250,ACTIVE
4,1234500000000,12/24 Mixing Tin Sets,Bar Tools,1234500000000,11.04,22.00,17.50,11.040,ACTIVE


In [71]:
begin.head()

,Customer,SKU,Category,Product,Quantity,Sale
0,Ah Sing Den,075496001602,Bitters,Angostura Bitters 16oz,1,16.9800
1,Ah Sing Den,088004143541,Liqueur,Pimms No 1 750ml,2,33.4800
2,Ah Sing Den,088004190163,Bitters,Regans Orange Bitters 5oz,2,9.9400
3,Ah Sing Den,088100915615,Vodka,Monopolowa Vodka 1L,5,63.7500
4,Ah Sing Den,123552,Bar Tools,Metal Straws,158,99.8268


In [72]:
end.head()

,Customer,SKU,Category,Product,Quantity,Sale
0,1417 French Bistro,080480004699,Liqueur,St Germain 750ml,9,266.40
1,1417 French Bistro,087236565008,Liqueur,Cointreau 1L,2,64.16
2,1417 French Bistro,182972001213,Rum,Clement Rhum Blanc Agricole 1L,2,41.66
3,1417 French Bistro,628451773108,Gin,Empress 1908 Gin 750ml,4,143.28
4,1417 French Bistro,674545000322,Tequila,Don Julio 1942 750ml,2,328.96


In [73]:
list_of_retail_stores = ['Customer Square ' + str(x) for x in range(1,4)] #we have 3 stores active, we won't count the Warehouse
retail_names = ['Airport','Pleasant Valley','Oak Hill'] # make sure this matches the above list length - no known automation possible
store_map = dict(zip(list_of_retail_stores,retail_names))

In [74]:
old_categories = begin.Category.unique()
new_categories = end.Category.unique()

cats = np.append(old_categories,new_categories)
cats = list(set(cats))
cats.sort()
#cats

In [75]:
inv_categories = inventory.Category.unique()
#inv_categories

In [76]:
#for cat in inv_categories:
#    if cat not in cats:
#        print(cat)

In [77]:
cats = list(set(cats + list(inv_categories)))
cats.sort()
#cats

In [78]:
wine_categories = ['Bubbles','Orange Wine','Port','Red', 'Red Beaujolais', 'Red Blend', 'Red Bordeaux', 'Red Cabernet Sauvignon', 'Red Chianti', 'Red Malbec', 'Red Merlot','Red Montepulciano', 'Red Mourvedre', 'Red New World', 'Red Old World', 'Red Pinot Noir', 'Red Primitivo', 'Red Tempranillo', 'Red Zinfandel','Rose','Sparkling rose','White','White Chardonnay','White Chenin','White Gewurztraminer','White Muscadet','White Picpoul','White Pinot Grigio','White Reisling','White Sauvignon Blanc','White wine']
#fucking hell, Seth
#len(wine_categories)

31

In [79]:
begin_wine = begin[begin.Category.isin(wine_categories) & (begin.Customer.isin(list_of_retail_stores))]

In [80]:
#end_wine = end[end.Category.isin(wine_categories) & (end.Customer.isin(list_of_retail_stores))]
end_wine = end[end.Category.isin(wine_categories) & (end.Customer.isin(['Customer Square 1','Customer Square 2']))]

In [81]:
begin_wine_profit = begin_wine.Sale.sum()
end_wine_profit = end_wine.Sale.sum()

ratio = end_wine_profit / begin_wine_profit
ratio

1.8382747387953506